In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import numpy as np
from birds_training import Dataset, BirdsNet, train_model
import torch

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
train_config ={
    'dataset': './dataset/bin_22050_32/',
    'file_stats': './files_std_data_32_bits.npy',
    'duration': 5,
    'SAMPLE_RATE': 22050,
    'pretrained_model': 'resnet18',
    'pretrained': True,
    'n_classes': 264,
    'n_mels': 128,
    'hop_size': 315,
    'window_size': 1024,
    'train_batch_size': 128,
    'val_batch_size': 128,
    'train_pin_memory': True,
    'val_pin_memory': True,
    'optimizer': 'Adam',
    'LEARNING_RATE': 0.001,
    'classes_file': './models/classes_264.npy',
    'train_files': './train_files_22050_264_32.npy',
    'val_files': './val_files_22050_264_32.npy',
    'N_samples_train': 2,
    'N_samples_val': 4,
    'f1_kaggle': '',
    'last_training_state': './models/librosa_model_train_state_mel_128_264_32_bw.pth',
    'best_model': './models/model_mel_264_v2_32.pth',
    'init_fourier': True,
    'init_mel': True,
    'train_fourier': False,
    'train_mel': True,
    'train_add_noise': True,
    'train_add_ambient_noise': True,
    'val_add_noise': False,
    'val_add_ambient_noise': False,
    'bytes_per_sample': 4,
    'mel_db': False,
    'a':0.0
}

- probar con hop size=315 y window_size=1024
- probar de no filtrar por amplitud ni sumar ruido
- Agrandar duración de entrenamiento (10 segs?)
- Entrenar con largo variable

In [6]:
model = BirdsNet(
    window_size=train_config['window_size'], 
    resnet_type=train_config['pretrained_model'], 
    pretrained=train_config['pretrained'],
    n_classes=train_config['n_classes'], 
    init_fourier=train_config['init_fourier'], 
    init_mel=train_config['init_mel'], 
    train_fourier=train_config['train_fourier'], 
    n_mels=train_config['n_mels'],
    hop_size=train_config['hop_size'],
    sr=train_config['SAMPLE_RATE'],
    mel_db=train_config['mel_db'],
    a=train_config['a']
).to(device)

Using cache found in /home/usuario/.cache/torch/hub/pytorch_vision_v0.6.0


In [7]:
classes = np.load(train_config['classes_file'])
file_stats = np.load(train_config['file_stats'], allow_pickle=True).item()
train_files = np.load(train_config['train_files'])
val_files = np.load(train_config['val_files'])

In [8]:
val_files_large = []
for file in val_files:
    if file_stats[file]['size'] >= train_config['SAMPLE_RATE']*train_config['duration']:
        val_files_large.append(file)

In [9]:
params_train = {
    'batch_size': train_config['train_batch_size'],
    'shuffle': True,
    'num_workers': 0,
    'pin_memory': train_config['train_pin_memory']
}

params_val = {
    'batch_size': train_config['val_batch_size'],
    'shuffle': True,
    'num_workers': 0,
    'timeout': 0,
    'pin_memory': train_config['val_pin_memory']
}


training_set = Dataset(
    list_IDs=list(train_files)*train_config['N_samples_train'], 
    classes=classes, 
    std_stats=file_stats, 
    chunk_seconds=train_config['duration'], 
    bytes_per_sample=train_config['bytes_per_sample'],
    sr=train_config['SAMPLE_RATE'], 
    add_noise=train_config['train_add_noise'], 
    add_ambient_noise=train_config['train_add_ambient_noise']
)

validation_set = Dataset(
    list_IDs=val_files_large*train_config['N_samples_val'], 
    classes=classes, 
    std_stats=file_stats, 
    chunk_seconds=train_config['duration'], 
    bytes_per_sample=train_config['bytes_per_sample'],
    sr=train_config['SAMPLE_RATE'],
    add_noise=train_config['val_add_noise'], 
    add_ambient_noise=train_config['val_add_ambient_noise']
)
training_generator = torch.utils.data.DataLoader(training_set, **params_train)
validation_generator = torch.utils.data.DataLoader(validation_set, **params_val)

In [10]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=train_config['LEARNING_RATE'])

In [12]:
print(model.window_size, model.resnet_type, model.pretrained, model.n_classes, model.init_fourier, 
    model.init_mel, 
    model.train_fourier, 
    model.n_mels,
    model.hop_size,
    model.sr,
    model.mel_db,
    model.a)

AttributeError: 'BirdsNet' object has no attribute 'window_size'

In [27]:
n_classes = train_config['n_classes']

epochs, losses, F1s, val_losses, val_F1s = train_model(
    model, 
    training_generator, 
    validation_generator, 
    criterion=criterion, 
    optimizer=optimizer,
    name=f'./models/mel_128_bw_a_1024_315/model_1024_315', 
    device=device,
    epochs=25
)

1/25 - 5/267 - loss: 0.40079107880592346, F1 micro: 0.00658178236335516, acc: 0.13770492374897003, 

KeyboardInterrupt: 

In [16]:
model.a

Parameter containing:
tensor([-1.1160], device='cuda:0', requires_grad=True)

In [17]:
# train_config['last_training_state']
# train_config['best_model'] = 'model_mel_v2_264_0.00943.pth'

In [27]:
torch.save({
    'epoch': 25,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, train_config['last_training_state'])

In [12]:
train_config['results'] = {}
train_config['results']['losses'] = losses
train_config['results']['val_losses'] = val_losses
train_config['results']['f1_score'] = F1s
train_config['results']['val_f1_score'] = val_F1s

In [15]:
# F1s_  = [F1.item() for F1 in F1s]
# losses_  = [vl.item() for vl in losses]

In [13]:
import time
np.save(f'training_configs/train_config_{int(time.time())}_mel_128_BW_a.npy', train_config)

In [46]:
print(training_set.stats)
print(validation_set.stats)

{'noise': 0, 'noise+signal': 0, 'signal': 0, 'passband_noise': 0, 'short_file': 0}
{'noise': 22700, 'noise+signal': 0, 'signal': 404700, 'passband_noise': 0, 'short_file': 22700}


In [11]:
# sd